In [291]:
import logging

# from ortools.linear_solver import pywraplp
import numpy as np
import pandas as pd
from pyomo.environ import *

In [292]:
np.random.seed(1)

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [293]:
LITURGY_TYPES = ['SUNDAY', 'DOM_LOW', 'BENED', 'SUNG', 'EXTRA_SOL', 'DOM_SOL']

SUNDAY_ROLES = ['MC', 'Thurifer', 'Crucifer', 'Acolyte']
OTHER_WEEKLY = ['Dominican_Low', 'Benediction']
DOM_SOL_ROLES = ['Senior_Acolyte', 'Junior_Acolyte', 'Crucifer', 'Thurifer']
ROLES = {
    'SUNDAY': SUNDAY_ROLES,
    'DOM_LOW': ['Dominican_Low'],
    'BENED': ['Benediction'],
    'SUNG': ['Sung_Mass'],
    'EXTRA_SOL': SUNDAY_ROLES,
    'DOM_SOL': DOM_SOL_ROLES    
}

N_SUNDAY_ROLES = len(SUNDAY_ROLES)
N_OTHER_WEEKLY = len(OTHER_WEEKLY)
N_DOMINICAN_SOLEMN = len(DOM_SOL_ROLES)

SUNDAY, DOM_LOW, BENED, SUNG, EXTRA_SOL, DOM_SOL = 'SUNDAY', 'DOM_LOW', 'BENED', 'SUNG', 'EXTRA_SOL', 'DOM_SOL'

In [294]:
SERVER_NAMES = [ # this constant will be useless after there's an actual availability table
    "Tim",
    "Seamus",
    "Luke",
    "Jack",
    "Rob",
    "Bernardo",
    "John_David",
    "Chao",
    "Jay",
    "Drey",
    "Thomas",
    "Josh",
    "Alex",
    "Mario",
    "Matthew",
    "Patrick",
    "Lord"
]

THURIFERS = set(SERVER_NAMES) - set(list([
    "Rob",
    "Josh",
    "Lord"
]))

LEAD_CANDLE_BEARERS = [
    "Tim",
    "Seamus",
    "Luke",
    "Jack",
    "Bernardo",
    "John_David",
    "Chao",
    "Jay",
    "Drey",
    "Alex",
    "Matthew"
]

MCS = [
    "Tim",
    "Seamus",
    "Luke",
    "Jack",
    "John_David",
    "Chao",
    "Drey"
]

n_servers = len(SERVER_NAMES)

Create availability table

In [295]:
n_weeks = 13
n_liturgies = n_weeks * 3 + 6 # (13 weeks * 3 weekly liturgies + random liturgies.). This constant will be useless after there's an actual availability table

sched_liturgy_types = [SUNDAY, BENED, DOM_LOW] * n_weeks
sched_liturgy_types.insert(5, SUNG)
sched_liturgy_types.insert(9, DOM_SOL)
sched_liturgy_types.insert(14, DOM_SOL)
sched_liturgy_types.insert(15, EXTRA_SOL)
sched_liturgy_types.insert(25, SUNG)
sched_liturgy_types.insert(35, DOM_SOL)
availability_data = np.random.random((len(SERVER_NAMES), n_liturgies)) < 0.95
lits_sched = [f'{ix:02d}_' + l for ix, l in enumerate(sched_liturgy_types)]
availability_data = pd.DataFrame(availability_data, columns=lits_sched, index=SERVER_NAMES)

Scheduling parameters. These can be modified if the solver is not converging.

In [296]:
min_break = 2
min_times_serving = 1
min_roles_served = 2 # has to be three or less
max_weekly = 2

Remove servers not to be scheduled for certain types of liturgies

In [297]:
for l, lt in enumerate(sched_liturgy_types):
    if lt == DOM_LOW:
        dl = np.array(sched_liturgy_types) == DOM_LOW
        availability_data.loc[["Rob", "Josh", "Alex", "Mario", "Patrick", "Lord"], dl] = False
    if lt in (SUNG, BENED):
        su = np.array(sched_liturgy_types) == SUNG
        be = np.array(sched_liturgy_types) == BENED
        sb = (su + be) > 0
        availability_data.loc[["Rob", "Bernardo", "John_David", "Thomas", "Josh", "Mario", "Patrick", "Lord"], sb] = False
    if lt == DOM_SOL:
        ds = np.array(sched_liturgy_types) == DOM_SOL
        availability_data.loc[["Rob", "John_David", "Drey", "Thomas", "Josh", "Alex", "Mario", "Patrick", "Lord"], ds] = False

In [298]:
n_servers = availability_data.shape[0]
n_liturgies = availability_data.shape[1]

Add decision variables. Each variable is a binary variable denoting whether or not a server is serving a liturgy in
a certain role.

In [299]:
model = ConcreteModel(name='Altar Server Scheduling')

for s, lits in availability_data.iterrows():
    # for each server, create a variable for each role in each liturgy
    server_components = []
    for l, avail in lits.items():
        # lit_no is the liturgy number, lit_type the time, and avail is whether the server is available for that liturgy
        lit_no, lit_type = l.split('_', 1)
        lit_no = int(lit_no)
        if avail:
            for r in ROLES[lit_type]:
                if r == 'MC' and s not in MCS:
                    # if the server is not an MC, he cannot serve as MC
                    model.add_component('x_%s_%s_%s' % (l, s, r), Var(domain={0}))
                    server_components.append('x_%s_%s_%s' % (l, s, r))
                    continue
                if r == 'Thurifer' and s not in THURIFERS:
                    # if the server is not a Thurifer, he cannot serve as Thurifer
                    model.add_component('x_%s_%s_%s' % (l, s, r), Var(domain={0}))
                    server_components.append('x_%s_%s_%s' % (l, s, r))
                    continue

                # if the server is available, he can serve in that role on that liturgy
                model.add_component('x_%s_%s_%s' % (l, s, r), Var(domain=Binary))
                server_components.append('x_%s_%s_%s' % (l, s, r))

            # for each server, create a constraint that he can only serve in one role for each liturgy
            model.add_component(
                'one_role_%s_%s' % (l, s), 
                Constraint(expr=sum(model.component('x_%s_%s_%s' % (l, s, r)) for r in ROLES[lit_type]) <= 1)
            )
        else:
            for r in ROLES[lit_type]:
                # if the server is not available, he cannot serve in that role on that liturgy
                model.add_component('x_%s_%s_%s' % (l, s, r), Var(domain={0}))

    # Add constraints to ensure each server serves at least n times.
    if len(server_components) > 0:
        model.add_component('min_serving_%s' % s, Constraint(
            expr=sum(model.component(c) for c in server_components) >= min_times_serving
        ))

Add constraints to make sure at the right number of servers in each roles is schedules to the liturgy

In [300]:
for l in availability_data.columns:
    # for each liturgy, create a constraint that each role is filled by the correct number of servers
    lit_no, lit_type = l.split('_', 1)
    if lit_type in ['SUNDAY', 'EXTRA_SOL']:
        model.add_component(
            'count_%s_%s' % (l, 'MC'), 
            Constraint(expr=sum(model.component('x_%s_%s_%s' % (l, s, 'MC')) for s in SERVER_NAMES) == 1)
        )
        model.add_component(
            'count_%s_%s' % (l, 'Thurifer'), 
            Constraint(expr=sum(model.component('x_%s_%s_%s' % (l, s, 'Thurifer')) for s in SERVER_NAMES) == 1)
        )
        model.add_component(
            'count_%s_%s' % (l, 'Crucifer'), 
            Constraint(expr=sum(model.component('x_%s_%s_%s' % (l, s, 'Crucifer')) for s in SERVER_NAMES) == 1)
        )
        model.add_component(
            'count_%s_%s' % (l, 'Acolyte'), 
            Constraint(expr=sum(model.component('x_%s_%s_%s' % (l, s, 'Acolyte')) for s in SERVER_NAMES) == 2)
        )
    elif lit_type == 'DOM_LOW':
        model.add_component(
            'count_%s_%s' % (l, 'Dominican_Low'), 
            Constraint(expr=sum(model.component('x_%s_%s_%s' % (l, s, 'Dominican_Low')) for s in SERVER_NAMES) == 1)
        )
    elif lit_type == 'SUNG':
        model.add_component(
            'count_%s_%s' % (l, 'Sung_Mass'), 
            Constraint(expr=sum(model.component('x_%s_%s_%s' % (l, s, 'Sung_Mass')) for s in SERVER_NAMES) == 1)
        )
    elif lit_type == 'DOM_SOL':
        model.add_component(
            'count_%s_%s' % (l, 'Senior_Acolyte'), 
            Constraint(expr=sum(model.component('x_%s_%s_%s' % (l, s, 'Senior_Acolyte')) for s in SERVER_NAMES) == 1)
        )
        model.add_component(
            'count_%s_%s' % (l, 'Junior_Acolyte'), 
            Constraint(expr=sum(model.component('x_%s_%s_%s' % (l, s, 'Junior_Acolyte')) for s in SERVER_NAMES) == 1)
        )
        model.add_component(
            'count_%s_%s' % (l, 'Crucifer'), 
            Constraint(expr=sum(model.component('x_%s_%s_%s' % (l, s, 'Crucifer')) for s in SERVER_NAMES) == 1)
        )
        model.add_component(
            'count_%s_%s' % (l, 'Thurifer'), 
            Constraint(expr=sum(model.component('x_%s_%s_%s' % (l, s, 'Thurifer')) for s in SERVER_NAMES) == 1)
        )
    elif lit_type == 'BENED':
        model.add_component(
            'count_%s_%s' % (l, 'Benediction'), 
            Constraint(expr=sum(model.component('x_%s_%s_%s' % (l, SERVER_NAMES[s], 'Benediction')) for s in range(n_servers)) == 1)
        )
    else:
        raise ValueError('Unknown liturgy type: %s' % lit_type)

Make sure at least one lead candle bearer is scheduled.

In [301]:
for l in availability_data.columns:
    # for each liturgy, create a constraint that each role is filled by the correct number of servers
    lit_no, lit_type = l.split('_', 1)
    if lit_type in ['SUNDAY', 'EXTRA_SOL']:
        model.add_component(
            'count_lead_acolyte_%s' % (l), 
            Constraint(expr=sum(model.component('x_%s_%s_%s' % (l, s, 'Acolyte')) for s in LEAD_CANDLE_BEARERS) >= 1)
        )

Make sure everybody is thurifer no more than once, crucifer no more than once, and candle bearer at least once.

In [302]:
max_thurifer = n_liturgies // (len(THURIFERS)) + 1
for s in SERVER_NAMES:
    comps_no_sol_thurifer = []
    comps_no_sol_acolyte = []
    comps_dom_low = []
    for l in lits_sched:
        if any(ll in l for ll in ['SUNDAY', 'EXTRA_SOL']):
            comps_no_sol_thurifer.append(model.component('x_%s_%s_%s' % (l, s, 'Thurifer')))
            comps_no_sol_acolyte.append(model.component('x_%s_%s_%s' % (l, s, 'Acolyte')))

    model.add_component(
        'serve_all_roles_%s_%s' % (s, 'Thurifer'),
        Constraint(expr=sum(comps_no_sol_thurifer) <= 2)
    )
    model.add_component(
        'serve_all_roles_%s_%s' % (s, 'Acolyte'),
        Constraint(expr=sum(comps_no_sol_acolyte) >= 1)
    )

Add constraints with auxiliary variables for the penalties in the objective function to minimize the discrepancies in the number of Masses served by each server.

In [303]:
model.add_component('b', Var(domain=PositiveIntegers))
for s1 in SERVER_NAMES:
    model.add_component('t_%s' % s1, Var(domain=PositiveIntegers))
    for s2 in SERVER_NAMES:
        if s1 == s2:
            continue
        model.add_component('d_%s_%s' % (s1, s2), Var(domain=Integers))

def sum_x(s):
    comps = []
    for l in lits_sched:
        for r in ROLES[l.split('_', 1)[1]]:
            comps.append(model.component('x_%s_%s_%s' % (l, s, r)))
    return sum(comps)

for s in SERVER_NAMES:
    model.add_component(
        'sum_served_%s' % s, 
        Constraint(expr=sum_x(s) == model.component('t_%s' % s))
    )

def sum_s1(s):
    comps = []
    for l in lits_sched:
        for r in ROLES[l.split('_', 1)[1]]:
            comps.append(model.component('x_%s_%s_%s' % (l, s, r)))
    return sum(comps)
    
for s in SERVER_NAMES:
    t_s = model.component('t_%s' % s)
    model.add_component(
        'sum_serve_%s' % s, 
        Constraint(expr=sum_s1(s) == t_s)
    )
        
for s1 in range(n_servers):
    for s2 in range(n_servers):
        if s1 == s2:
            continue
        t_s1 = model.component('t_%s' % SERVER_NAMES[s1])
        t_s2 = model.component('t_%s' % SERVER_NAMES[s2])

        d_s1_s2 = model.component('d_%s_%s' % (SERVER_NAMES[s1], SERVER_NAMES[s2]))

        model.add_component(
            'diff_total_served_%s_%s' % (SERVER_NAMES[s1], SERVER_NAMES[s2]),
            Constraint(expr=d_s1_s2 == t_s1 - t_s2)
        )
        model.add_component(
            'diff_total_served_%s_%s_abs' % (SERVER_NAMES[s1], SERVER_NAMES[s2]),
            Constraint(expr=model.component('b') >= d_s1_s2)
        )

Add constraints with auxiliary variables for the penalties in the objective function to minimize the number of weeks a server serves in a row.

In [304]:
model.add_component('m', Var(domain=PositiveReals)) # should be PositiveIntegers
for s in SERVER_NAMES:
    for ln in range(n_liturgies - min_break):
        lit_name = lits_sched[ln]
        model.add_component('v_%s_%s' % (lit_name, s), Var(domain=PositiveReals)) # should be PositiveIntegers

for s in SERVER_NAMES:
    for ln in range(n_liturgies - min_break):
        lit_name = lits_sched[ln]
        aux_sum_served = model.component('v_%s_%s' % (lit_name, s))
        
        comp_served_interval = []
        for ln2 in range(ln, ln + min_break + 1):
            lit_name = lits_sched[ln2]
            lit_no, lit_type = lit_name.split('_', 1)
            for r in ROLES[lit_type]:
                c = model.component('x_%s_%s_%s' % (lit_name, s, r))
                comp_served_interval.append(c)

        model.add_component(
            'served_that_week_%s_%s' % (lit_name, s),
            Constraint(expr=sum(comp_served_interval) == aux_sum_served)
        )
        model.add_component(
            'served_that_week_%s_%s_abs' % (lit_name, s),
            Constraint(expr=model.component('m') >= aux_sum_served)
        )
        

Add objective function that will have penalties for (1) large discrepancies in the number of Masses served by each server, and for (2) servers serving too many times in a row.

In [305]:
M_frequency_one, M_frequency_discrepandy = 1000, 1000
model.obj = Objective(expr=M_frequency_discrepandy * model.component('b') + M_frequency_one * model.component('m'), sense=minimize)
# model.obj = Objective(expr=M_frequency_discrepandy * model.component('b'), sense=minimize)

In [306]:
mps_path = "model.mps"

# Assuming 'model' is your Pyomo model
model.write(mps_path, format='mps', io_options={'symbolic_solver_labels': True})

from ortools.linear_solver.python import model_builder
from ortools.linear_solver import pywraplp

model_ortools = model_builder.ModelBuilder()
model_ortools.import_from_mps_file(mps_path)

solver = model_builder.ModelSolver('SCIP')
solver.enable_output(True)
status = solver.solve(model_ortools)

print("Problem name: ", model_ortools.name)
print("Number of variables: ", model_ortools.num_variables)
print("Objective value: ", solver.objective_value)
print("Number of constraints: ", model_ortools.num_constraints)

if status == pywraplp.Solver.INFEASIBLE:
    print("The problem is infeasible.")
elif status == pywraplp.Solver.OPTIMAL:
    print("An optimal solution has been found.")


Problem name:  Altar
Number of variables:  2654
Objective value:  1999.9999999999982
Number of constraints:  2718
An optimal solution has been found.


In [307]:
from pprint import pprint


variables = solver.values(model_ortools.get_variables())
variables_1 = variables.loc[variables == 1]
variables_1.index = [v.name for v in variables_1.index]

df = serving_df = pd.DataFrame('', index=SERVER_NAMES, columns=lits_sched)
res = {}
for lit in lits_sched:
    roles = {}
    for r in ROLES[lit.split('_', 1)[1]]:
        for s in SERVER_NAMES:
            if f'x_{lit}_{s}_{r}' in variables_1.index:
                if r == 'Acolyte' and 'Acolyte 1' not in roles:
                    roles['Acolyte 1'] = s
                elif r == 'Acolyte' and 'Acolyte 1' in roles:
                    roles['Acolyte 2'] = s
                else:
                    roles[r] = s
                df.loc[s, lit] = r
    res[lit] = roles

pprint(res, indent=4)
with open('roles.json', 'w') as f:
    import json
    json.dump(res, f, indent=4)
print('\n', df.to_string())
print('\n', (df != '').sum(axis=1))

{   '00_SUNDAY': {   'Acolyte 1': 'John_David',
                     'Acolyte 2': 'Lord',
                     'Crucifer': 'Josh',
                     'MC': 'Drey',
                     'Thurifer': 'Seamus'},
    '01_BENED': {'Benediction': 'Alex'},
    '02_DOM_LOW': {'Dominican_Low': 'Thomas'},
    '03_SUNDAY': {   'Acolyte 1': 'Bernardo',
                     'Acolyte 2': 'Mario',
                     'Crucifer': 'Patrick',
                     'MC': 'John_David',
                     'Thurifer': 'Matthew'},
    '04_BENED': {'Benediction': 'Luke'},
    '05_SUNG': {'Sung_Mass': 'Chao'},
    '06_DOM_LOW': {'Dominican_Low': 'Matthew'},
    '07_SUNDAY': {   'Acolyte 1': 'Rob',
                     'Acolyte 2': 'Drey',
                     'Crucifer': 'Josh',
                     'MC': 'Jack',
                     'Thurifer': 'Alex'},
    '08_BENED': {'Benediction': 'Luke'},
    '09_DOM_SOL': {   'Crucifer': 'Jay',
                      'Junior_Acolyte': 'Bernardo',
                     

In [308]:
print(variables_1.loc[['v_' in v for v in variables_1.index]].min())

1.0
